In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
import pandas as pd

In [3]:
names = []
prices = []
urls = []

In [4]:
with webdriver.Chrome() as webpage:
    webpage.get('https://lista.mercadolivre.com.br/iphone#D[A:iphone]')
    
    ol = WebDriverWait(webpage, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="root-app"]/div/div[3]/section/ol'))
    )
    
    for _ in range(6):
        webpage.execute_script('window.scrollTo(0,document.body.scrollHeight);')

        lis = WebDriverWait(webpage, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="root-app"]/div/div[3]/section/ol/li'))
        )

        for li in lis:
            name = li.find_element(By.XPATH, './/h2[contains(@class, "ui-search-item__title")]')
            names.append(name.text)
            price = li.find_element(By.XPATH, './/span[2][contains(@class, "andes-money-amount__fraction")]')
            prices.append(price.text)
            url = li.find_element(By.XPATH, './/a[1][contains(@class, "ui-search-link")]').get_attribute('href')
            urls.append(url)
            sleep(0.2)
            
        next_page_button = WebDriverWait(webpage, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="root-app"]/div/div[3]/section/nav/ul/li[last()]/a'))
        )
        next_page_button.click()
        sleep(2)

In [6]:
handled_prices = []
for price in prices:
    handled_prices.append(int(price.replace('.','')))

In [7]:
ml_analysis_df = pd.DataFrame(data=[names, handled_prices, urls]).T
ml_analysis_df.columns = ['title', 'price', 'link']

In [8]:
ml_analysis_df.head(10)

,title,price,link
0,Apple iPhone 14 (128 GB) - Estelar - Distribui...,5499,https://www.mercadolivre.com.br/apple-iphone-1...
1,Apple iPhone 13 (128 GB) - Estelar - Distribui...,7599,https://www.mercadolivre.com.br/apple-iphone-1...
2,Apple iPhone 13 (128 GB) Meia-noite - Distribu...,7599,https://www.mercadolivre.com.br/apple-iphone-1...
3,Apple iPhone 14 (128 GB) - Meia noite,5499,https://www.mercadolivre.com.br/apple-iphone-1...
4,Apple iPhone 15 (128 GB) - Preto - Distribuido...,5319,https://www.mercadolivre.com.br/apple-iphone-1...
5,Apple iPhone 14 Plus (128 Gb) - Estelar - Dist...,4670,https://www.mercadolivre.com.br/apple-iphone-1...
6,Apple iPhone 14 (128 GB) - Roxo - Distribuidor...,5499,https://www.mercadolivre.com.br/apple-iphone-1...
7,Apple iPhone 12 (128 GB) - Preto - Distribuido...,3999,https://www.mercadolivre.com.br/apple-iphone-1...
8,Apple iPhone 14 Plus (128 GB) - Meia-noite - D...,4670,https://www.mercadolivre.com.br/apple-iphone-1...
9,Apple iPhone 15 (128 GB) - Amarelo - Distribui...,7438,https://www.mercadolivre.com.br/apple-iphone-1...


In [9]:
ml_analysis_df.isna().sum()

title    0
price    0
link     0
dtype: int64

In [10]:
ml_analysis_df.shape

(292, 3)

In [11]:
ml_analysis_df.to_csv('ml_analysis_df.csv', index=False)